## Submitted By:
# Abhinav Rai
# 72111354

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from skimage.color import rgb2gray
import matplotlib.pyplot as plt

### For Image 1 (Variation 1)

In [ ]:
image_1 = cv2.imread('wall.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

In [ ]:

#resized_image_1 = cv2.resize(image_1, (600, 550))
#cv2_imshow(resized_image_1)

#grayscale = rgb2gray(resized_image_1)

#grayscale = cv2.cvtColor(image_1, cv2.COLOR_RGB2BGR)



#plt.imshow(image_1)
print(image_1.shape)

# Noise Reduction

In [ ]:
#noise_red = cv2.bilateralFilter(image_1, 4, 8, 8)
#noise_red = cv2.medianBlur(image_1, 5)
noise_red = cv2.GaussianBlur(image_1, (9, 9 ), 9)

displaying = np.concatenate((image_1, noise_red), axis=1)
cv2_imshow(displaying)

# Gradient Calculation for image_1

In [ ]:
g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)

g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)

g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)




''' NOT WORKED AS EXPECTED
fig, axes = plt.subplots(1, 3, figsize=(26, 20))
ax = axes.ravel()

ax[0].imshow(g_x)
ax[0].set_title("Horizontally emphazied")
ax[1].imshow(g_y)
ax[1].set_title("Vertically emphazied")
ax[2].imshow(g_xy)
ax[2].set_title("Both emphazied")
fig.tight_layout()
plt.show()'''

In [ ]:
#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

print(gradient_magnitude)
print(gradient_direction)

In [ ]:
displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

# NMS --> Non Maximum Suppression

In [ ]:
def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

## Double Threshold

In [ ]:
#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

# Edge tracking by hysteresis

In [ ]:
#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 160

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

In [ ]:
'''edge_tracked = cv2.dilate(double_Threshold, None)
edge_tracked = cv2.erode(double_Threshold, None)'''

In [ ]:
'''cv2_imshow(double_Threshold) #displaying the final result
cv2.waitKey(0)
cv2.destroyAllWindows()'''

## Variation 2 (increase in brightness, contrast of picture)

In [ ]:
image_1 = cv2.imread('wall.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_1.shape)

inc_brightness= cv2.convertScaleAbs(image_1, alpha=2.5, beta=2.5)
#cv2_imshow(inc_brightness)

noise_red = cv2.GaussianBlur(inc_brightness, (9, 9 ), 9)
displaying = np.concatenate((image_1,inc_brightness, noise_red), axis=1)
cv2_imshow(displaying)

g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255


#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 160

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

## Variation 3 , Decreasing brightness and checking the result

In [ ]:
image_1 = cv2.imread('wall.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_1.shape)

dec_brightness = cv2.convertScaleAbs(image_1, alpha=0.5, beta=0)

#cv2_imshow(dec_brightness)

noise_red = cv2.GaussianBlur(dec_brightness, (9, 9 ), 7)
displaying = np.concatenate((image_1, dec_brightness, noise_red), axis=1)
cv2_imshow(displaying)

g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

# For Image 2 (Variation 1)

In [ ]:
image_2 = cv2.imread('side tree.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_2.shape)

noise_red = cv2.GaussianBlur(image_2, (7, 7 ), 7)
displaying = np.concatenate((image_2, noise_red), axis=1)
cv2_imshow(displaying)

g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

# Variation --> 2 , Rotating image

In [ ]:

image_2 = cv2.imread('side tree.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_2.shape)
Rotation = cv2.rotate(image_2, cv2.ROTATE_90_CLOCKWISE)  #rotating image on 90 degree
#cv2_imshow(Rotation)

noise_red = cv2.GaussianBlur(Rotation, (5, 5 ), 5)
displaying = np.concatenate((Rotation, noise_red), axis=1)
cv2_imshow(displaying)

g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)


#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

# Variation -->3 , increasing the brightness

In [ ]:

image_2 = cv2.imread('side tree.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_2.shape)
inc_brightness= cv2.convertScaleAbs(image_2, alpha=2.5, beta=2.5)
#cv2_imshow(inc_brightness)

noise_red = cv2.GaussianBlur(inc_brightness, (5, 5 ), 5)
displaying = np.concatenate((image_2, inc_brightness, noise_red), axis=1)
cv2_imshow(displaying)

g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

# For Image 3 (Variation 1)

In [ ]:

image_3 = cv2.imread('b_back.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_3.shape)


noise_red = cv2.GaussianBlur(image_3, (7, 7 ), 7)
displaying = np.concatenate((image_3, noise_red), axis=1)
cv2_imshow(displaying)

g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)



#Threshold Values
T1_low = 60
T2_high = 160

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255


edge_tracked = cv2.dilate(double_Threshold, None)
edge_tracked = cv2.erode(double_Threshold, None)

cv2_imshow(double_Threshold) #displaying the final result
cv2.waitKey(0)
cv2.destroyAllWindows()

## Variation -->2 , decreasing contrast

In [ ]:
image_3 = cv2.imread('b_back.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_3.shape)
Low_contrast = cv2.convertScaleAbs(image_3, alpha=1.0, beta=-30)
#cv2_imshow(Low_contrast)

noise_red = cv2.GaussianBlur(Low_contrast, (7, 7 ), 7)
displaying = np.concatenate((image_3, Low_contrast, noise_red), axis=1)
cv2_imshow(displaying)

g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

## Variation -->3, increasing the contrast and check output

In [ ]:
image_3 = cv2.imread('b_back.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_3.shape)
inc_contrast = cv2.convertScaleAbs(image_3, alpha=1.0, beta=100)
#cv2_imshow(Low_contrast)

noise_red = cv2.GaussianBlur(inc_contrast, (7, 7 ), 7)
displaying = np.concatenate((image_3, inc_contrast, noise_red), axis=1)
cv2_imshow(displaying)

g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

# For Image 4 (Variation 1)

In [ ]:
image_4 = cv2.imread('cycles.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_4.shape)

noise_red = cv2.GaussianBlur(image_4, (7, 7 ), 7)
displaying = np.concatenate((image_4, noise_red), axis=1)
cv2_imshow(displaying)

g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

## Variation 2, increasing brightness and rotating in 90 degree

In [ ]:
image_4 = cv2.imread('cycles.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_4.shape)

inc_brightness = cv2.convertScaleAbs(image_4, alpha=2.5, beta=0)

rotated_image = cv2.rotate(inc_brightness, cv2.ROTATE_90_CLOCKWISE)

noise_red = cv2.GaussianBlur(rotated_image, (7, 7 ), 7)
displaying = np.concatenate((image_4, inc_brightness), axis=1)
cv2_imshow(displaying)

cv2_imshow(rotated_image)

g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

## Variation 3 ---> decreasing brightness and increasing the contrast of the image

In [ ]:
image_4 = cv2.imread('cycles.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_4.shape)

dec_brightness = cv2.convertScaleAbs(image_4, alpha=0.5, beta=0)

inc_contrast = cv2.convertScaleAbs(dec_brightness, alpha=1.0, beta=50)

noise_red = cv2.GaussianBlur(inc_contrast, (7, 7 ), 7)
displaying = np.concatenate((image_4, dec_brightness, inc_contrast, noise_red), axis=1)
cv2_imshow(displaying)


g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

## Now for image 5 (Variation 1)

In [ ]:
image_5 = cv2.imread('side_space_area.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_5.shape)

noise_red = cv2.GaussianBlur(image_5, (7, 7 ), 7)
displaying = np.concatenate((image_5, noise_red), axis=1)
cv2_imshow(displaying)


g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

# Variation -->2 resizing the image and checking the result

In [ ]:
image_5 = cv2.imread('side_space_area.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_5.shape)
cv2_imshow(image_5)

resized_image = cv2.resize(image_5, (650, 650))


noise_red = cv2.GaussianBlur(resized_image, (5, 5 ), 3)
displaying = np.concatenate((resized_image, noise_red), axis=1)
cv2_imshow(displaying)


g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output

## Variation --> 3 (Resizing + increasing brightness + inc contrast)

In [ ]:
image_5 = cv2.imread('side_space_area.jpeg', 0)
#The 0 argument in the code specifies that the output image should be in grayscale

print(image_5.shape)
cv2_imshow(image_5)

resized_image = cv2.resize(image_5, (650, 650))
inc_brightness= cv2.convertScaleAbs(resized_image, alpha=2.5, beta=2.5)

noise_red = cv2.GaussianBlur(inc_brightness, (5, 5 ), 3)
displaying = np.concatenate((resized_image, inc_brightness, noise_red), axis=1)
cv2_imshow(displaying)


g_x = cv2.Sobel(noise_red, cv2.CV_64F, 1, 0, ksize=3)
g_y = cv2.Sobel(noise_red, cv2.CV_64F, 0, 1, ksize=3)
g_xy = cv2.addWeighted(g_x,  0.5, g_y, 0.5, 0)

displaying_sobel_images = np.concatenate((g_x, g_y, g_xy), axis=1)
cv2_imshow(displaying_sobel_images)

#1 image is g_x in which edges are emphazied horizontally
#2 image is g_y in which edges are emphazied vertically
#3 image is g_xy in which edges are emphazied  both (horizontally and Vertically)

#Calculating the magnitude of g_x and g_y
gradient_magnitude = np.sqrt(g_x**2 + g_y**2)

#Calculating the Gradient orientation, used to identify the direction of the edge
gradient_direction = np.arctan2(g_y, g_x) * (180 / np.pi) % 180

displaying_g_images = np.concatenate((noise_red, gradient_magnitude, gradient_direction), axis=1)
cv2_imshow(displaying_g_images)

def NMS(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)

    for m in range(1, rows - 1):
        for n in range(1, cols - 1):
            angle_gm = gradient_direction[m, n]

# now we will find the neighboring pixels by gradient direction
            if (0 <= angle_gm < 25) or (160 <= angle_gm <= 180):
                neighbour = [gradient_magnitude[m, n+1], gradient_magnitude[m, n-1]]
            elif 25 <= angle_gm < 70:
                neighbour = [gradient_magnitude[m-1, n+1], gradient_magnitude[m+1, n-1]]
            elif 70 <= angle_gm < 115:
                neighbour = [gradient_magnitude[m-1, n], gradient_magnitude[m+1, n]]
            else:
                neighbour = [gradient_magnitude[m-1, n-1], gradient_magnitude[m+1, n+1]]

            # Perform non-maximum suppression
            if gradient_magnitude[m, n] >= max(neighbour):
                suppressed[m, n] = gradient_magnitude[m, n]

    return suppressed

suppressed = NMS(gradient_magnitude, gradient_direction)

#double Threshold for identifying edges
double_Threshold = np.zeros_like(suppressed)
double_Threshold[(suppressed >= T1_low) & (suppressed <= T2_high)] = 255

#edge tracking by hysteresis
def edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high):
    rows, cols = double_Threshold.shape
    E_visited = np.zeros((rows, cols), dtype=np.uint8)

    def track_edge(m, n):
        if m < 0 or m >= rows or n < 0 or n >= cols or E_visited[m, n]:
            return
        E_visited[m, n] = 255
        if T1_low <= double_Threshold[m, n] <= T2_high:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    track_edge(m + dx, n + dy)
    for m in range(rows):
        for n in range(cols):
            if double_Threshold[m, n] > T2_high and not E_visited[m, n]:
                track_edge(m, n)

    return E_visited

#Threshold Values
T1_low = 60
T2_high = 180

result = edge_tracked_by_hysteresis(double_Threshold, T1_low, T2_high)

cv2_imshow(result) #to display the final output